In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import display
import os, sys, itertools, csv
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from util.alemutdf import get_all_sample_mut_df, get_multi_exp_max_freq_mut_df
from util.mut import is_coding_mut
from util.metadata import get_condition_val_dict, get_condition_field_val_set
from util.genome import get_K12_pos_from_BOP27

In [2]:
muts_df = get_all_sample_mut_df("./data/muts/published/")
# unpublished_muts_df = get_all_sample_mut_df("./data/muts/unpublished/")  # I've currently placed GYD into the public mutation set.
# muts_df = muts_df.append(unpublished_muts_df)
display(set(muts_df["exp"]))

  0%|          | 0/29 [00:00<?, ?it/s]

./data/muts/published//Central carbon knockout glucose evolution.csv
./data/muts/published//42C.csv
./data/muts/published//PGI.csv
./data/muts/published//Central carbon knockout gnd.csv
./data/muts/published//SSW GLU XYL.csv


100%|██████████| 4/4 [00:00<00:00, 259.49it/s]


./data/muts/published//SER.csv
./data/muts/published//SSW GLU AC.csv
./data/muts/published//SSW AC.csv
./data/muts/published//GLU.csv


 41%|████      | 27/66 [00:00<00:00, 267.40it/s]

...

./data/muts/published//Central carbon knockout PTS.csv
./data/muts/published//SSW GLY.csv
./data/muts/published//SSW XYL.csv
./data/muts/published//Central carbon knockout tpiA.csv
./data/muts/published//GYD.csv


100%|██████████| 12/12 [00:00<00:00, 266.39it/s]

./data/muts/published//Central carbon knockout sdh.csv
./data/muts/published//C13.csv



/home/pphaneuf/ava_ms/ava/util/alemutdf.py:106: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  mutat_df = pd.concat(mutat_df_list)


{'42C',
 'C13',
 'Central_carbon_knockout_PTS',
 'Central_carbon_knockout_glucose_evolution',
 'Central_carbon_knockout_gnd',
 'Central_carbon_knockout_pgi',
 'Central_carbon_knockout_sdh',
 'Central_carbon_knockout_tpiA',
 'GLU',
 'GYD',
 'PGI',
 'SER',
 'SSW_AC',
 'SSW_GLU_AC',
 'SSW_GLU_GLY',
 'SSW_GLU_XYL',
 'SSW_GLY',
 'SSW_XYL'}

In [3]:
l = [
    "42c_tenaillon",
    "ltee",
    "ltee_ara",
    "tee",
    "tee_ara",
    "tm_glugly",
    "epistasis2011",
    'C321',
    'C321.∆A',
    'C321.∆A.earlyfix',
    'ECNR2.1',
    'Auxotrophic_strains_hisD_gltA',
    'Auxotrophic_strains_hisD_gltB',
    'Auxotrophic_strains_hisD_pyrC',
]
exp_to_remove_l = [x.lower() for x in l]
muts_df = muts_df[~muts_df.exp.str.lower().isin(exp_to_remove_l)]
set(muts_df.exp)

{'42C',
 'C13',
 'Central_carbon_knockout_PTS',
 'Central_carbon_knockout_glucose_evolution',
 'Central_carbon_knockout_gnd',
 'Central_carbon_knockout_pgi',
 'Central_carbon_knockout_sdh',
 'Central_carbon_knockout_tpiA',
 'GLU',
 'GYD',
 'PGI',
 'SER',
 'SSW_AC',
 'SSW_GLU_AC',
 'SSW_GLU_GLY',
 'SSW_GLU_XYL',
 'SSW_GLY',
 'SSW_XYL'}

## remove problem and starting strain mutations

In [4]:
muts_df = muts_df[~(
    ((muts_df.Gene == "gatC")
     | (muts_df.Gene == "gatC, gatC"))
    & ((muts_df.Position == "2,173,361")
       | (muts_df.Position == "2,173,363")
       | (muts_df.Position == "2,173,364"))
    & ((muts_df["Sequence Change"] == "Δ2 bp")
       | (muts_df["Sequence Change"] == "Δ1 bp"))
)]

muts_df = muts_df[~(
    (muts_df.Gene == "[crl]")
    & (muts_df.Position == "257,908")
    & (muts_df["Sequence Change"] == "Δ776 bp"))]

muts_df = muts_df[~(
    (muts_df.Gene == "insB1, insA")
    & (muts_df.Position == "1,978,503")
    & (muts_df["Sequence Change"] == "Δ776 bp"))]

muts_df = muts_df[~(
    (muts_df.Gene == "ychE, oppA")
    & (muts_df.Position == "1,299,499")
    & (muts_df["Sequence Change"] == "Δ1,199 bp"))]

muts_df = muts_df[~(
    ((muts_df.Gene == "glpR")
     | (muts_df.Gene == "glpR, glpR"))
    & (muts_df.Position == "3,560,455")
    & (muts_df["Sequence Change"] == "+G"))]

muts_df = muts_df[~(
    (muts_df.Gene == "gltP, yjcO")
    & (muts_df.Position == "4,296,381")
    & (muts_df["Sequence Change"] == "+GC"))]

muts_df = muts_df[~(
    (muts_df.Gene == "gltP, yjcO")
    & (muts_df.Position == "4,296,380")
    & (muts_df["Sequence Change"] == "+CG"))]

muts_df = muts_df[~(
    (muts_df.Gene == "insB1,insA")
    & (muts_df.Position == "1,978,503")
    & (muts_df["Sequence Change"] == "Δ776 bp"))]

muts_df = muts_df[~(
    ((muts_df.Gene == "insA/uspC") | (muts_df.Gene == "insA, uspC"))
    & (muts_df.Position == "1,979,486")
    & (muts_df["Sequence Change"] == "IS5 (+) +4 bp"))]

p = {'4,293,212',
 '4,293,403',
 '4,293,430',
 '4,296,060',
 '4,296,154',
 '4,296,189',
 '4,296,190',
 '4,296,191',
 '4,296,267',
 '4,296,268',
 '4,296,286',
 '4,296,380',
 '4,296,382'}
ch = {'+C', '+G', 'A→C', 'A→G', 'A→T', 'C→A', 'C→T', 'T→C'}
muts_df = muts_df[~(
    (muts_df.Gene == "gltP, yjcO")
    & (muts_df["Position"].isin(p))
    & (muts_df["Sequence Change"].isin(ch))
       )]

muts_df.head()

,Details,Gene,Mut ID,Mutation Type,Position,Reference Seq,Sequence Change,ale,exp,flask,isolate,presence,tech_rep
1,K286R (AAG→AGG),insH1,NaN,SNP,"2,066,465",NaN,T→C,23,Central_carbon_knockout_glucose_evolution,0,0,0.37,1
2,I204S (ATC→AGC),wcaA,NaN,SNP,"2,132,787",NaN,A→C,23,Central_carbon_knockout_glucose_evolution,0,0,1.00,1
3,,"[rph], [rph]",NaN,DEL,"3,815,858",NaN,Δ82 bp,23,Central_carbon_knockout_glucose_evolution,0,0,1.00,1
4,coding (220‑224/951 nt),corA,NaN,DEL,"4,001,644",NaN,Δ5 bp,23,Central_carbon_knockout_glucose_evolution,0,0,1.00,1
5,P1100Q (CCG→CAG),rpoB,NaN,SNP,"4,184,542",NaN,C→A,23,Central_carbon_knockout_glucose_evolution,0,0,1.00,1


## Remove experiment specific starting strain mutations

In [5]:
# Filter the following mutations out if they come from specific experiments.
muts_df = muts_df[(~((muts_df.exp=="SER") & (muts_df["Gene"]=="cyoB") & (muts_df["Details"]=="W190* (TGG→TAG)")))]
muts_df = muts_df[(~((muts_df.exp=="SER") & (muts_df["Gene"]=="eno") & (muts_df["Details"]=="L193Q (CTG→CAG)")))]
muts_df = muts_df[(~((muts_df.exp=="SER") & (muts_df["Gene"]=="chbF") & (muts_df["Details"]=="V133E (GTA→GAA)")))]
muts_df = muts_df[(~((muts_df.exp=="SER") & (muts_df["Gene"]=="fumC") & (muts_df["Details"]=="G354G (GGT→GGC)")))]
muts_df = muts_df[(~((muts_df.exp=="SER") & (muts_df["Gene"]=="yegE") & (muts_df["Details"]=="S683Y (TCC→TAC)")))]
muts_df = muts_df[(~((muts_df.exp=="SER") & (muts_df["Gene"]=="rcsC") & (muts_df["Details"]=="L594F (TTG→TTC)")))]


muts_df = muts_df[(~((muts_df.exp=="GYD") & (muts_df["Gene"]=="fumC") & (muts_df["Details"]=="G354G (GGT→GGC)")))]
muts_df = muts_df[(~((muts_df.exp=="GYD") & (muts_df["Gene"]=="yegE") & (muts_df["Details"]=="S683Y (TCC→TAC)")))]
muts_df = muts_df[(~((muts_df.exp=="GYD") & (muts_df["Gene"]=="rcsC") & (muts_df["Details"]=="L594F (TTG→TTC)")))]


cck_exp_list = [
    "Central_carbon_knockout_tpiA",
    "Central_carbon_knockout_PTS",
    "Central_carbon_knockout_pgi",
    "Central_carbon_knockout_sdh",
    "Central_carbon_knockout_gnd",
    "Central_carbon_knockout_glucose_evolution"
]

muts_df = muts_df[~(
    (muts_df.exp.isin(cck_exp_list))
    & (
        (muts_df["Gene"]=="wcaA")
        | (muts_df["Gene"]=="corA")
    )
)]  # The details of the mutations to filter out should probably include the sequence change.

In [6]:
# Since the majority of our comparisons have to do with endpoints, just look at endpoints.
# If not limiting to endpoint, the mutations used in associated mutations to conditions will need filtering.
# This occurs in a later NB of the pipeline.
muts_df = get_multi_exp_max_freq_mut_df(muts_df, endpoint_flask_only=True)
muts_df.head()

,Details,Gene,Mut ID,Mutation Type,Position,Reference Seq,Sequence Change,ale,exp,flask,isolate,presence,tech_rep
2,R110G (CGT→GGT),clsA,NaN,SNP,"1,308,318",NaN,G→C,1,42C,124,1,1.0,1
6,,"[rph], [rph]",NaN,DEL,"3,815,859",NaN,Δ82 bp,1,42C,124,1,1.0,1
7,A734V (GCG→GTG),rpoC,NaN,SNP,"4,187,550",NaN,C→T,1,42C,124,1,1.0,1
8,D9A (GAT→GCT),hfq,NaN,SNP,"4,400,313",NaN,A→C,1,42C,124,1,1.0,1
0,coding (380‑400/1149 nt),nagA,NaN,DEL,"702,352",NaN,Δ21 bp,1,42C,124,1,1.0,1


In [7]:
muts_df.Position = muts_df.Position.apply(lambda x: int(x.replace(",","")))

In [8]:
# ensure all mutations are using NC_000913 positions.
genome_pos_transform_exp_l = ["ERS",
                              'Central_carbon_knockout_PTS',
                              'Central_carbon_knockout_glucose_evolution',
                              'Central_carbon_knockout_gnd',
                              'Central_carbon_knockout_pgi',
                              'Central_carbon_knockout_sdh',
                              'Central_carbon_knockout_tpiA',]
muts_df.Position = muts_df.apply(lambda row: get_K12_pos_from_BOP27(row.Position) if row.exp in genome_pos_transform_exp_l else row.Position, axis=1)

In [9]:
muts_df["coding"] = muts_df.Details.apply(is_coding_mut)
muts_df["Gene"] = muts_df.apply(lambda row: row["Gene"].replace(", ","/") if not row["coding"] else row["Gene"], axis=1)

muts_df.head()

,Details,Gene,Mut ID,Mutation Type,Position,Reference Seq,Sequence Change,ale,exp,flask,isolate,presence,tech_rep,coding
2,R110G (CGT→GGT),clsA,NaN,SNP,1308318,NaN,G→C,1,42C,124,1,1.0,1,True
6,,"[rph], [rph]",NaN,DEL,3815859,NaN,Δ82 bp,1,42C,124,1,1.0,1,True
7,A734V (GCG→GTG),rpoC,NaN,SNP,4187550,NaN,C→T,1,42C,124,1,1.0,1,True
8,D9A (GAT→GCT),hfq,NaN,SNP,4400313,NaN,A→C,1,42C,124,1,1.0,1,True
0,coding (380‑400/1149 nt),nagA,NaN,DEL,702352,NaN,Δ21 bp,1,42C,124,1,1.0,1,True


In [10]:
print(len(muts_df))
muts_df = muts_df[muts_df.presence >= 0.5]
print(len(muts_df))

1798
1732


In [11]:
# This work is also currently duplicated NB4 
muts_df["Gene"] = muts_df["Gene"].apply(lambda a: "rph" if a == "[rph], [rph]" else a)
muts_df["Gene"] = muts_df["Gene"].apply(lambda a: "rph" if a == "[rph],[rph]" else a)
muts_df.head()

,Details,Gene,Mut ID,Mutation Type,Position,Reference Seq,Sequence Change,ale,exp,flask,isolate,presence,tech_rep,coding
2,R110G (CGT→GGT),clsA,NaN,SNP,1308318,NaN,G→C,1,42C,124,1,1.0,1,True
6,,rph,NaN,DEL,3815859,NaN,Δ82 bp,1,42C,124,1,1.0,1,True
7,A734V (GCG→GTG),rpoC,NaN,SNP,4187550,NaN,C→T,1,42C,124,1,1.0,1,True
8,D9A (GAT→GCT),hfq,NaN,SNP,4400313,NaN,A→C,1,42C,124,1,1.0,1,True
0,coding (380‑400/1149 nt),nagA,NaN,DEL,702352,NaN,Δ21 bp,1,42C,124,1,1.0,1,True


In [12]:
# removing unnecessary column
muts_df = muts_df.drop(columns=["Mut ID"])

In [13]:
# Amount of ALEs before removal
print(len(muts_df.apply(lambda r: r.exp + " " + str(r.ale), axis=1).unique()))

# removing hypermutators since they will affect associations

# GLU hypermutators
muts_df = muts_df[
    ~(
        (muts_df.exp == "GLU")
        & ((muts_df.ale == 5) | (muts_df.ale == 7))
    )
]
print(len(muts_df.apply(lambda r: r.exp + " " + str(r.ale), axis=1).unique()))

125
123


In [14]:
# removing samples not incuded in GYD MS for AVA MS

muts_df = muts_df[
    ~(
        (muts_df.exp == "GYD")
        & ((muts_df.ale == 35) | (muts_df.ale == 34))
    )
]
print(len(muts_df.apply(lambda r: r.exp + " " + str(r.ale), axis=1).unique()))

121


In [15]:
muts_df.to_pickle("./data/1_df.pkl")

In [16]:
display(len(muts_df.exp.unique()))

18

In [17]:
for e in muts_df.exp.unique():
    print(e)

42C
C13
Central_carbon_knockout_PTS
Central_carbon_knockout_glucose_evolution
Central_carbon_knockout_gnd
Central_carbon_knockout_pgi
Central_carbon_knockout_sdh
Central_carbon_knockout_tpiA
GLU
GYD
PGI
SER
SSW_AC
SSW_GLU_AC
SSW_GLU_GLY
SSW_GLU_XYL
SSW_GLY
SSW_XYL


In [18]:
muts_df["exp ALE"] = muts_df.apply(lambda r: r.exp + ' ' + str(r.ale), axis=1)
display(len(muts_df["exp ALE"].unique()), muts_df["exp ALE"].unique())

121

array(['42C 1', '42C 2', '42C 3', '42C 4', '42C 5', '42C 6', '42C 7',
       '42C 8', '42C 9', '42C 10', 'C13 1', 'C13 2', 'C13 3', 'C13 4',
       'C13 5', 'C13 6', 'Central_carbon_knockout_PTS 0',
       'Central_carbon_knockout_PTS 9', 'Central_carbon_knockout_PTS 10',
       'Central_carbon_knockout_PTS 11', 'Central_carbon_knockout_PTS 12',
       'Central_carbon_knockout_glucose_evolution 23',
       'Central_carbon_knockout_glucose_evolution 24',
       'Central_carbon_knockout_gnd 0', 'Central_carbon_knockout_gnd 17',
       'Central_carbon_knockout_gnd 18', 'Central_carbon_knockout_gnd 19',
       'Central_carbon_knockout_pgi 0', 'Central_carbon_knockout_pgi 1',
       'Central_carbon_knockout_pgi 2', 'Central_carbon_knockout_pgi 3',
       'Central_carbon_knockout_pgi 4', 'Central_carbon_knockout_pgi 5',
       'Central_carbon_knockout_pgi 6', 'Central_carbon_knockout_pgi 7',
       'Central_carbon_knockout_pgi 8', 'Central_carbon_knockout_pgi 9',
       'Central_carbon_knock

In [19]:
len(muts_df)

1278